In [1]:
import pandas as pd
from sklearn import linear_model, preprocessing, model_selection, metrics, ensemble
import random
import tqdm

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)


In [42]:
df = pd.read_csv('/media/td/Samsung_T5/sports/nba/feature_reduction5.csv', sep = '|')
df.sort_values('win_loss_accuracy', ascending = False).head()

,player_aggregate_stat,team_aggregate_stat,aggregation_windows,team_aggregation_functions,player_aggregation_functions,use_team_data,use_player_data,manual_entry,win_loss_accuracy,points_estimation_r2,col_count
0,"['fg3a_per_fga_pct', 'fg_pct', 'ft_pct', 'fta', 'ast', 'mp', 'tov', 'pts', 'usg_pct', 'fg', 'stl', 'ast_pct', 'orb_pct', 'pf', 'drb', 'orb', 'blk', 'fga', 'ts_pct', 'blk_pct', 'fta_per_fga_pct', 'def_rtg', 'drb_pct', 'stl_pct', 'fg3', 'trb_pct', 'fg3_pct', 'plus_minus']","['def_rtg', 'fg3a', 'ast', 'fga', 'drb_pct', 'fta', 'score_diff', 'win', 'pf', 'trb', 'blk', 'orb_pct', 'fg3', 'ft', 'blk_pct', 'fg3_pct', 'trb_pct', 'pts', 'orb', 'stl', 'tov', 'usg_pct', 'stl_pct', 'ts_pct', 'ast_pct', 'days_since_last_match', 'efg_pct']","[75, 1, 100, 7, 3]","['min', 'avg']","['min', 'avg', 'var']",1,1,1,0.672904,0.186384,1504
2,"['pts', 'tov', 'stl_pct', 'fg3', 'fg3a_per_fga_pct', 'tov_pct', 'usg_pct', 'ft', 'orb', 'ts_pct', 'fg3_pct', 'blk_pct', 'ast', 'ast_pct', 'orb_pct', 'drb_pct', 'mp', 'off_rtg', 'fta', 'trb_pct', 'fga', 'ft_pct', 'fg_pct', 'fg', 'efg_pct', 'def_rtg', 'fg3a', 'stl', 'trb', 'blk', 'plus_minus', 'drb']","['fta_per_fga_pct', 'fga', 'win']","[7, 3, 75]",['avg'],"['median', 'max', 'avg', 'min', 'var', 'skew']",1,1,1,0.671551,0.187284,1156
1,"['ast', 'orb_pct', 'plus_minus', 'ft', 'drb_pct', 'stl_pct', 'blk_pct', 'ft_pct', 'tov_pct', 'blk', 'fg', 'fg3', 'trb', 'ts_pct', 'fg3a_per_fga_pct', 'pts']","['trb', 'tov_pct', 'ast_pct', 'off_rtg', 'ts_pct', 'ft_pct', 'orb', 'fta_per_fga_pct', 'stl', 'ast', 'ft', 'pts', 'days_since_last_match', 'efg_pct', 'def_rtg', 'fga', 'stl_pct', 'fg3_pct', 'tov', 'fg3a_per_fga_pct', 'score_diff', 'fg3', 'fta', 'drb_pct', 'blk_pct']","[100, 50, 1, 75, 40, 12, 7, 30, 5, 3]",['avg'],"['max', 'var', 'median', 'avg', 'skew']",1,1,1,0.671551,0.187159,1610
3,"['fga', 'off_rtg', 'def_rtg', 'drb', 'trb', 'fta_per_fga_pct', 'usg_pct', 'fg3a', 'plus_minus', 'blk', 'tov_pct', 'ast_pct', 'fg3_pct', 'fg3a_per_fga_pct', 'blk_pct', 'stl', 'efg_pct', 'ft']","['efg_pct', 'blk_pct', 'ft_pct', 'fg3_pct', 'fta_per_fga_pct']","[75, 7, 50, 100]",['avg'],"['avg', 'max', 'median', 'var']",1,1,1,0.670875,0.190023,586
4,"['fg3a_per_fga_pct', 'fg3', 'fta_per_fga_pct', 'fta', 'blk', 'fg_pct', 'orb_pct', 'ast', 'drb_pct', 'efg_pct', 'pf', 'drb', 'tov', 'fg', 'plus_minus', 'fga', 'mp', 'stl_pct', 'blk_pct', 'stl', 'fg3_pct']","['score_diff', 'days_since_last_match', 'efg_pct', 'fg3a_per_fga_pct', 'usg_pct', 'blk', 'ast', 'pts', 'fg3', 'fg', 'orb_pct', 'tov_pct', 'def_rtg', 'off_rtg', 'drb_pct', 'orb', 'drb', 'pf', 'ast_pct', 'fg3a', 'fg3_pct', 'stl_pct', 'ft', 'fta', 'stl', 'fga', 'trb_pct', 'ft_pct', 'home']","[50, 7, 12, 3, 75, 40]","['avg', 'max']","['max', 'var', 'avg', 'skew']",1,1,1,0.670198,0.186535,2026


In [3]:
baseline_classification =  pd.read_csv('/tmp/baseline_classification.csv', sep = '|')
baseline_classification.head()

,column,feature_importance,slope,intercept,r_value,r2_value,p_value,std_err,model_performance,normalized_feature_importance,aggregation_window_size,team_aggregation_function,player_aggregation_function,stat_type,diff_feature,feature_type
0,team_pregame_rating_0_diff_vs_opponent_feature,15849.583517,0.000174,0.500000,0.318186,0.101242,0.000000e+00,0.000004,0.663886,0.262325,NaN,NaN,NaN,NaN,1,general
1,home_diff_vs_opponent_feature,4090.004925,0.087889,0.500000,0.173594,0.030135,4.500050e-120,0.003743,0.663886,0.067693,NaN,NaN,NaN,NaN,1,general
2,home,3817.787024,0.171498,0.415846,0.171468,0.029401,3.731829e-117,0.007398,0.663886,0.063188,NaN,NaN,NaN,NaN,0,general
3,team_aggregate_past_75_game_median_player_stats_aggregated_by_game_plus_minus_avg_diff_vs_opponent_feature,2103.318169,0.061730,0.500000,0.326820,0.106812,0.000000e+00,0.001340,0.663886,0.034812,75.0,median,avg,plus_minus,1,player_aggregate
4,team_aggregate_past_75_game_avg_player_stats_aggregated_by_game_plus_minus_avg_diff_vs_opponent_feature,1622.311670,0.072195,0.500000,0.332520,0.110569,0.000000e+00,0.001537,0.663886,0.026851,75.0,avg,avg,plus_minus,1,player_aggregate


In [4]:

player_aggregate_stat_choices = ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct',
                                   'efg_pct', 'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct',
                                   'fg_pct', 'fga', 'ft', 'ft_pct', 'fta', 'fta_per_fga_pct',
                                   'mp', 'off_rtg', 'orb', 'orb_pct', 'pf', 'plus_minus', 'pts',
                                   'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct', 'ts_pct',
                                   'usg_pct']

team_aggregate_stat_choices = ['ast', 'ast_pct', 'blk', 'blk_pct', 'def_rtg', 'drb', 'drb_pct', 'efg_pct',
                                      'fg', 'fg3', 'fg3_pct', 'fg3a', 'fg3a_per_fga_pct', 'fg_pct', 'fga', 'ft',
                                      'ft_pct', 'fta', 'fta_per_fga_pct', 'off_rtg', 'orb', 'orb_pct', 'pf',
                                      'pts', 'stl', 'stl_pct', 'tov', 'tov_pct', 'trb', 'trb_pct',
                                      'ts_pct', 'usg_pct', 'home', 'win', 'days_since_last_match', 'score_diff']

aggregation_window_sizes = [1, 3, 5, 7, 12, 20, 30, 40, 50, 75, 100]
team_aggregation_functions = ['avg', 'var', 'skew', 'max', 'median', 'min']
player_aggregation_functions = ['avg', 'var', 'skew', 'max', 'median', 'min']


def get_random_params_and_col_list(feature_importance_df):
#     use_team_data = random.choice([1, 0])
#     use_player_data = random.choice([1, 0])
    use_team_data = 1
    use_player_data = 1
    
    player_aggregate_stat_num = random.randint(1, len(player_aggregate_stat_choices))
    team_aggregate_stat_num = random.randint(1, len(team_aggregate_stat_choices))
    aggregation_window_size_num = random.randint(1, len(aggregation_window_sizes))
    team_aggregation_function_num = random.randint(1, len(team_aggregation_functions))
    player_aggregation_function_num = random.randint(1, len(player_aggregation_functions))
    
    player_aggregate_stat_final_choice = random.sample(player_aggregate_stat_choices, player_aggregate_stat_num)
    team_aggregate_stat_final_choice = random.sample(team_aggregate_stat_choices, team_aggregate_stat_num)
    aggregation_window_final_choice = random.sample(aggregation_window_sizes, aggregation_window_size_num)
    team_aggregation_function_final_choice = random.sample(team_aggregation_functions, team_aggregation_function_num)
    player_aggregation_function_final_choice = random.sample(player_aggregation_functions, player_aggregation_function_num)
    
#     player_aggregate_stat_final_choice = player_aggregate_stat_choices
#     team_aggregate_stat_final_choice = team_aggregate_stat_choices
#     aggregation_window_final_choice = aggregation_window_sizes
#     team_aggregation_function_final_choice = team_aggregation_functions
#     player_aggregation_function_final_choice = player_aggregation_functions
    
    
    feature_importance_df_team = feature_importance_df[feature_importance_df['feature_type'] == 'team_aggregate']
    feature_importance_df_player = feature_importance_df[feature_importance_df['feature_type'] == 'player_aggregate']
    feature_importance_df_general = feature_importance_df[feature_importance_df['feature_type'] == 'general']
    
    output_columns = feature_importance_df_general['column'].tolist()
    
    team_cols = feature_importance_df_team[(feature_importance_df_team['stat_type'].isin(team_aggregate_stat_final_choice))&
                          (feature_importance_df_team['aggregation_window_size'].isin(aggregation_window_final_choice))&
                          (feature_importance_df_team['team_aggregation_function'].isin(team_aggregation_function_final_choice))]['column'].tolist()
    
    player_cols = feature_importance_df_player[(feature_importance_df_player['stat_type'].isin(player_aggregate_stat_final_choice))&
                          (feature_importance_df_player['aggregation_window_size'].isin(aggregation_window_final_choice))&
                          (feature_importance_df_player['team_aggregation_function'].isin(team_aggregation_function_final_choice))&\
                           (feature_importance_df_player['player_aggregation_function'].isin(player_aggregation_function_final_choice))]['column'].tolist()
    
    if use_team_data:
        output_columns.extend(team_cols)
    if use_player_data:
        output_columns.extend(player_cols)
    return {'player_aggregate_stat':player_aggregate_stat_final_choice,
           'team_aggregate_stat':team_aggregate_stat_final_choice,
           'aggregation_windows':aggregation_window_final_choice,
           'team_aggregation_functions':team_aggregation_function_final_choice,
           'player_aggregation_functions':player_aggregation_function_final_choice,
           'output_columns':output_columns,
           'use_team_data':use_team_data,
           'use_player_data':use_player_data}


In [5]:
recs = []

counter = 0
while len(recs) < 1000000:
    counter += 1
    if counter %1000 == 0:
        print(counter, len(recs))
    
    tmp = get_random_params_and_col_list(baseline_classification)
#     print(len(tmp['output_columns']))
    if len(tmp['output_columns']) < 12 or len(tmp['output_columns']) > 4000:
        continue
    
    rec = dict()
    rec['col_count'] = len(tmp['output_columns'])
    rec['use_player_data'] = tmp['use_player_data']
    rec['use_team_data'] = tmp['use_team_data']
    rec['aggregation_windows'] = tmp['aggregation_windows']
    rec['player_aggregate_stat'] = tmp['player_aggregate_stat']
    rec['player_aggregation_functions'] = tmp['player_aggregation_functions']
    rec['team_aggregate_stat'] = tmp['team_aggregate_stat']
    rec['team_aggregation_functions'] = tmp['team_aggregation_functions']
    recs.append(rec)
    

1000 796
2000 1609
3000 2422
4000 3232
5000 4060
6000 4892
7000 5715
8000 6514
9000 7316
10000 8108
11000 8917
12000 9730
13000 10536
14000 11360
15000 12161
16000 12978
17000 13764
18000 14579
19000 15371
20000 16168
21000 16977
22000 17798
23000 18635
24000 19422
25000 20237
26000 21049
27000 21879
28000 22695
29000 23486
30000 24275
31000 25114
32000 25916
33000 26708
34000 27531
35000 28339
36000 29174
37000 29978
38000 30819
39000 31623
40000 32422
41000 33230
42000 34039
43000 34842
44000 35640
45000 36441
46000 37267
47000 38090
48000 38905
49000 39710
50000 40530
51000 41324
52000 42116
53000 42923
54000 43721
55000 44525
56000 45335
57000 46138
58000 46941
59000 47746
60000 48566
61000 49385
62000 50185
63000 50996
64000 51796
65000 52607
66000 53419
67000 54208
68000 55036
69000 55829
70000 56648
71000 57463
72000 58292
73000 59103
74000 59897
75000 60687
76000 61499
77000 62304
78000 63096
79000 63915
80000 64733
81000 65535
82000 66342
83000 67154
84000 67977
85000 68779
86

604000 489080
605000 489898
606000 490696
607000 491489
608000 492303
609000 493125
610000 493936
611000 494769
612000 495581
613000 496382
614000 497200
615000 498032
616000 498862
617000 499638
618000 500452
619000 501264
620000 502070
621000 502875
622000 503692
623000 504513
624000 505305
625000 506112
626000 506943
627000 507748
628000 508581
629000 509404
630000 510223
631000 511040
632000 511836
633000 512640
634000 513443
635000 514239
636000 515061
637000 515864
638000 516677
639000 517489
640000 518308
641000 519103
642000 519907
643000 520709
644000 521513
645000 522332
646000 523156
647000 523957
648000 524775
649000 525594
650000 526411
651000 527231
652000 528056
653000 528880
654000 529678
655000 530487
656000 531290
657000 532102
658000 532912
659000 533728
660000 534526
661000 535318
662000 536112
663000 536931
664000 537717
665000 538503
666000 539300
667000 540131
668000 540949
669000 541758
670000 542582
671000 543391
672000 544212
673000 545003
674000 545816
675000

1177000 952644
1178000 953444
1179000 954239
1180000 955058
1181000 955841
1182000 956641
1183000 957442
1184000 958266
1185000 959066
1186000 959872
1187000 960692
1188000 961492
1189000 962307
1190000 963127
1191000 963948
1192000 964759
1193000 965537
1194000 966346
1195000 967143
1196000 967967
1197000 968781
1198000 969592
1199000 970401
1200000 971208
1201000 972023
1202000 972845
1203000 973657
1204000 974485
1205000 975285
1206000 976085
1207000 976908
1208000 977709
1209000 978543
1210000 979347
1211000 980138
1212000 980960
1213000 981769
1214000 982580
1215000 983352
1216000 984179
1217000 984969
1218000 985786
1219000 986570
1220000 987380
1221000 988201
1222000 989031
1223000 989853
1224000 990665
1225000 991480
1226000 992290
1227000 993111
1228000 993914
1229000 994720
1230000 995517
1231000 996342
1232000 997157
1233000 997968
1234000 998778
1235000 999599


In [6]:
df_rec = pd.DataFrame.from_dict(recs)
df_rec.to_csv('/media/td/Samsung_T5/sports/nba/df_rec.csv', index = False, sep = '|')

df_rec = pd.read_csv('/media/td/Samsung_T5/sports/nba/df_rec.csv', sep = '|')
df_rec.head()

,col_count,use_player_data,use_team_data,aggregation_windows,player_aggregate_stat,player_aggregation_functions,team_aggregate_stat,team_aggregation_functions
0,1450,1,1,"[7, 20, 100, 5, 30, 50]","['blk', 'fg3', 'blk_pct', 'stl_pct', 'drb', 'off_rtg', 'usg_pct', 'ast', 'tov_pct', 'tov', 'drb_pct', 'fg3a_per_fga_pct', 'fg3_pct', 'def_rtg', 'trb', 'fta', 'pf', 'pts', 'fg_pct', 'ts_pct']",['skew'],"['ts_pct', 'blk', 'ft_pct', 'home', 'fg', 'tov_pct', 'fta', 'fta_per_fga_pct', 'trb', 'orb_pct', 'drb_pct']","['median', 'skew', 'var', 'max', 'min', 'avg']"
1,1132,1,1,"[3, 12, 75, 40, 20, 1, 5, 7, 30]","['fg3_pct', 'fg3a', 'tov_pct', 'blk_pct', 'pts', 'trb_pct', 'orb_pct', 'fta_per_fga_pct', 'stl', 'ts_pct', 'drb', 'def_rtg', 'off_rtg', 'orb', 'fta', 'ast', 'stl_pct']",['max'],"['days_since_last_match', 'orb', 'efg_pct', 'fta_per_fga_pct', 'fg_pct', 'fga', 'fta', 'fg']","['avg', 'skew', 'median', 'var']"
2,1726,1,1,"[30, 75, 100, 5, 50, 40, 1, 3, 7]","['trb', 'mp', 'tov_pct', 'tov', 'fg3_pct', 'usg_pct', 'stl_pct', 'ast_pct', 'def_rtg', 'ts_pct', 'ft_pct', 'fga', 'drb_pct', 'pts', 'blk_pct', 'fta', 'fg3', 'fg', 'orb', 'plus_minus', 'ft', 'pf', 'orb_pct', 'blk', 'efg_pct', 'fta_per_fga_pct']",['skew'],"['orb', 'tov_pct', 'home', 'trb', 'trb_pct', 'fta_per_fga_pct', 'win', 'score_diff', 'fta', 'fg3a_per_fga_pct', 'ft', 'fga', 'tov', 'fg3', 'fg', 'usg_pct', 'fg3a', 'ts_pct', 'blk', 'ft_pct', 'blk_pct', 'stl', 'pts', 'ast_pct']","['skew', 'avg', 'min', 'var']"
3,1438,1,1,"[3, 100, 40, 30, 5, 12, 20]","['fta', 'tov', 'usg_pct', 'blk', 'def_rtg', 'mp', 'ts_pct', 'fg3_pct', 'fg3', 'plus_minus', 'stl_pct', 'stl', 'fg_pct', 'trb', 'ft_pct', 'fg', 'efg_pct', 'off_rtg', 'orb', 'ft', 'trb_pct', 'fga', 'fta_per_fga_pct', 'ast', 'pf', 'orb_pct']","['min', 'skew']","['fg3a', 'ast', 'blk_pct', 'home', 'drb', 'ft_pct', 'fta_per_fga_pct', 'drb_pct', 'orb', 'win', 'fg3', 'trb_pct', 'pts', 'fga', 'off_rtg', 'fg_pct', 'fg', 'orb_pct', 'blk', 'usg_pct', 'stl', 'tov_pct', 'ts_pct', 'trb', 'fg3a_per_fga_pct', 'pf', 'ft', 'def_rtg']","['max', 'skew']"
4,202,1,1,"[12, 75, 50, 100, 30, 1, 40]","['ast_pct', 'stl']","['skew', 'avg', 'median', 'max']","['win', 'def_rtg', 'orb', 'ft', 'fg_pct', 'pts', 'days_since_last_match', 'fta', 'tov_pct', 'stl', 'trb_pct', 'fga', 'stl_pct', 'fg3', 'fg3_pct']","['var', 'max']"


In [7]:

df_rec.shape

(1000000, 8)

In [43]:
df = pd.read_csv('/media/td/Samsung_T5/sports/nba/feature_reduction.csv', sep = '|')
df2 = pd.read_csv('/media/td/Samsung_T5/sports/nba/feature_reduction2.csv', sep = '|')
df3 = pd.read_csv('/media/td/Samsung_T5/sports/nba/feature_reduction3.csv', sep = '|')
df4 = pd.read_csv('/media/td/Samsung_T5/sports/nba/feature_reduction4.csv', sep = '|')
df5 = pd.read_csv('/media/td/Samsung_T5/sports/nba/feature_reduction5.csv', sep = '|')
df = pd.concat([df, df2, df3, df4])
df.sort_values('win_loss_accuracy', ascending = False).head(20)


/home/td/anaconda3/envs/sports_predictor/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,aggregation_windows,col_count,manual_entry,player_aggregate_stat,player_aggregation_functions,points_estimation_r2,team_aggregate_stat,team_aggregation_functions,use_player_data,use_team_data,win_loss_accuracy
0,"[40, 50, 1, 7, 30, 3, 75]",1256,1.0,"['tov', 'fg3a', 'fg3a_per_fga_pct', 'trb', 'stl', 'blk', 'def_rtg', 'ts_pct', 'plus_minus', 'drb_pct', 'off_rtg', 'fg3', 'fg', 'orb_pct', 'fta']","['median', 'max', 'var', 'skew', 'avg', 'min']",0.186301,"['fg', 'usg_pct', 'fg3', 'ast_pct', 'days_since_last_match', 'efg_pct', 'tov', 'fg3_pct', 'fta', 'blk', 'fga', 'ast', 'drb_pct', 'win', 'pf', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'stl', 'fg_pct', 'home', 'trb_pct', 'fg3a', 'tov_pct', 'orb', 'orb_pct', 'off_rtg']",['avg'],1,1,0.672904
0,"[75, 7]",366,1.0,"['drb', 'ast', 'trb_pct', 'blk_pct', 'drb_pct', 'stl_pct', 'fg3', 'fg3a', 'fg3_pct', 'orb_pct', 'ts_pct', 'fta', 'off_rtg', 'plus_minus', 'tov_pct', 'ast_pct', 'ft_pct', 'tov']","['avg', 'min', 'max', 'var', 'skew']",0.186476,"['fg3_pct', 'efg_pct', 'fga', 'fg3a_per_fga_pct', 'stl_pct', 'fg_pct', 'pf', 'drb_pct', 'days_since_last_match', 'fta', 'tov_pct', 'orb_pct', 'fg3a', 'fg3', 'ft_pct', 'win', 'orb', 'blk_pct', 'ft', 'ast', 'def_rtg', 'usg_pct', 'ast_pct', 'off_rtg', 'drb', 'trb_pct', 'ts_pct']",['avg'],1,1,0.672002
1,"[1, 7, 75]",460,1.0,"['tov_pct', 'tov', 'usg_pct', 'orb', 'ast_pct', 'blk_pct', 'ts_pct', 'fg', 'pf', 'trb_pct', 'fg_pct', 'ft', 'mp', 'fg3_pct', 'fta_per_fga_pct', 'plus_minus', 'fta', 'orb_pct', 'ast']","['avg', 'var', 'min', 'skew']",0.187934,"['blk_pct', 'ft', 'fta_per_fga_pct', 'home', 'trb_pct']",['avg'],1,1,0.670649
1,"[7, 1, 75]",622,1.0,"['mp', 'plus_minus', 'blk', 'fta_per_fga_pct', 'fg3a_per_fga_pct', 'stl', 'pf', 'orb', 'trb_pct', 'pts', 'ast_pct', 'fga', 'ast', 'off_rtg', 'ts_pct', 'trb', 'fg3', 'usg_pct', 'efg_pct', 'fg3_pct', 'stl_pct', 'blk_pct', 'fg_pct', 'drb_pct', 'drb', 'fg3a', 'ft', 'fta', 'tov_pct', 'def_rtg', 'orb_pct', 'tov', 'fg', 'ft_pct']","['avg', 'var', 'skew']",0.187917,"['pts', 'fta', 'tov_pct', 'fg3_pct', 'fg', 'ts_pct', 'fg_pct', 'efg_pct', 'ast', 'orb', 'orb_pct', 'trb_pct', 'home', 'tov', 'blk', 'off_rtg', 'def_rtg', 'pf', 'blk_pct', 'score_diff', 'fta_per_fga_pct', 'drb']",['avg'],1,1,0.670198
2,"[3, 75, 30, 7, 5, 1]",2408,1.0,"['plus_minus', 'stl_pct', 'fta', 'blk', 'ast_pct', 'pts', 'drb', 'tov', 'efg_pct', 'fg3', 'fg', 'orb_pct', 'orb', 'ft', 'ast', 'fta_per_fga_pct', 'fg3a', 'ft_pct', 'tov_pct', 'off_rtg', 'trb', 'blk_pct']","['var', 'median', 'min', 'max', 'avg']",0.185791,"['fg3_pct', 'fg3a']","['avg', 'skew']",1,1,0.669748
2,"[7, 75]",754,1.0,"['fta_per_fga_pct', 'fg3a_per_fga_pct', 'usg_pct', 'ast_pct', 'ft', 'ast', 'blk', 'tov', 'pf', 'pts', 'drb_pct', 'fg3', 'orb_pct', 'fg3_pct', 'orb', 'stl', 'fga', 'efg_pct', 'blk_pct', 'fg_pct', 'mp', 'fg3a', 'tov_pct', 'plus_minus', 'trb_pct', 'ts_pct', 'def_rtg', 'drb', 'stl_pct', 'off_rtg', 'fta']","['var', 'skew', 'avg']",0.181047,"['fg3_pct', 'pf', 'blk', 'fg3a', 'blk_pct', 'stl_pct', 'off_rtg', 'def_rtg', 'stl', 'ft', 'score_diff', 'usg_pct', 'ast', 'orb', 'trb_pct', 'drb', 'fta', 'home', 'fta_per_fga_pct', 'ts_pct', 'efg_pct', 'fg_pct', 'orb_pct', 'win', 'drb_pct', 'trb', 'ft_pct', 'fga']","['max', 'avg']",1,1,0.669748
0,"[7, 1, 75, 30, 40]",1858,1.0,"['fg3a_per_fga_pct', 'trb_pct', 'pts', 'pf', 'fg3_pct', 'orb', 'fg3', 'off_rtg', 'plus_minus', 'ts_pct', 'fg3a']","['max', 'var', 'avg', 'skew']",0.189013,"['off_rtg', 'drb', 'pf', 'fg', 'tov', 'efg_pct', 'win', 'fta', 'trb_pct']","['max', 'avg', 'min', 'skew', 'median']",1,1,0.669297
3,"[7, 3, 75, 1]",1362,1.0,"['blk_pct', 'stl_pct', 'tov', 'orb', 'plus_minus', 'fg3a', 'mp', 'fga', 'efg_pct', 'pf', 'ast_pct', 'ts_pct', 'fg3a_per_fga_pct', 'def_rtg', 'orb_pct', 'fg_pct', 'off_rtg', 'drb', 'fg3', 'tov_pct', 'ft_pct', 'usg_pct', 'stl', 'fg', 'fta_per_fga_pct', 'ft', 'pts', 'fg3_pct', 'drb_pct', 'trb_pct', 'ast', 'trb', 'blk', 'fta']","['max', 'skew', 'avg', 'min', 'median']",0.185869,"['trb_pct', 'tov', 'pts', 'win', 'fg3', '

In [9]:
df.shape


(5326, 11)

In [10]:

outcome_accuracy_baseline = 0.6638863841298467
r2_baseline = 0.18689847911073687

df['_outcome_accuracy_difference_from_baseline'] = df['win_loss_accuracy'] - outcome_accuracy_baseline
df['_score_r2_difference_from_baseline'] = df['points_estimation_r2'] - r2_baseline

df = df.reindex(sorted(df.columns), axis=1)


In [11]:

df[(df['_outcome_accuracy_difference_from_baseline'] > 0)&(df['_score_r2_difference_from_baseline'] > 0)].sort_values('_outcome_accuracy_difference_from_baseline', ascending = False).head(20)


,_outcome_accuracy_difference_from_baseline,_score_r2_difference_from_baseline,aggregation_windows,col_count,manual_entry,player_aggregate_stat,player_aggregation_functions,points_estimation_r2,team_aggregate_stat,team_aggregation_functions,use_player_data,use_team_data,win_loss_accuracy
1,0.006763,0.001035,"[1, 7, 75]",460,1.0,"['tov_pct', 'tov', 'usg_pct', 'orb', 'ast_pct', 'blk_pct', 'ts_pct', 'fg', 'pf', 'trb_pct', 'fg_pct', 'ft', 'mp', 'fg3_pct', 'fta_per_fga_pct', 'plus_minus', 'fta', 'orb_pct', 'ast']","['avg', 'var', 'min', 'skew']",0.187934,"['blk_pct', 'ft', 'fta_per_fga_pct', 'home', 'trb_pct']",['avg'],1,1,0.670649
1,0.006312,0.001019,"[7, 1, 75]",622,1.0,"['mp', 'plus_minus', 'blk', 'fta_per_fga_pct', 'fg3a_per_fga_pct', 'stl', 'pf', 'orb', 'trb_pct', 'pts', 'ast_pct', 'fga', 'ast', 'off_rtg', 'ts_pct', 'trb', 'fg3', 'usg_pct', 'efg_pct', 'fg3_pct', 'stl_pct', 'blk_pct', 'fg_pct', 'drb_pct', 'drb', 'fg3a', 'ft', 'fta', 'tov_pct', 'def_rtg', 'orb_pct', 'tov', 'fg', 'ft_pct']","['avg', 'var', 'skew']",0.187917,"['pts', 'fta', 'tov_pct', 'fg3_pct', 'fg', 'ts_pct', 'fg_pct', 'efg_pct', 'ast', 'orb', 'orb_pct', 'trb_pct', 'home', 'tov', 'blk', 'off_rtg', 'def_rtg', 'pf', 'blk_pct', 'score_diff', 'fta_per_fga_pct', 'drb']",['avg'],1,1,0.670198
0,0.005410,0.002115,"[7, 1, 75, 30, 40]",1858,1.0,"['fg3a_per_fga_pct', 'trb_pct', 'pts', 'pf', 'fg3_pct', 'orb', 'fg3', 'off_rtg', 'plus_minus', 'ts_pct', 'fg3a']","['max', 'var', 'avg', 'skew']",0.189013,"['off_rtg', 'drb', 'pf', 'fg', 'tov', 'efg_pct', 'win', 'fta', 'trb_pct']","['max', 'avg', 'min', 'skew', 'median']",1,1,0.669297
1,0.005185,0.003604,"[75, 7, 1, 3, 50]",940,1.0,"['ft_pct', 'tov_pct', 'fg3_pct', 'blk_pct', 'off_rtg', 'orb_pct', 'pf', 'ast', 'ft', 'trb', 'fg3a', 'tov', 'drb_pct', 'plus_minus', 'drb', 'stl_pct', 'def_rtg', 'orb', 'fg3a_per_fga_pct', 'mp', 'fga', 'efg_pct', 'usg_pct', 'ast_pct', 'ts_pct', 'fta_per_fga_pct', 'fg_pct', 'fg', 'stl', 'pts', 'fg3']","['max', 'avg', 'var']",0.190502,"['pts', 'orb', 'home', 'stl_pct', 'pf', 'fg3a', 'blk_pct', 'fg_pct', 'trb_pct', 'orb_pct', 'fg', 'fga', 'tov_pct', 'off_rtg', 'win', 'efg_pct']",['avg'],1,1,0.669071
4,0.005185,0.001176,"[3, 75, 7]",766,1.0,"['pts', 'ft', 'pf', 'fg', 'fta', 'tov_pct', 'off_rtg', 'trb', 'trb_pct', 'plus_minus', 'ast_pct', 'ft_pct', 'fg3', 'blk_pct', 'orb_pct', 'ts_pct', 'drb', 'efg_pct', 'def_rtg', 'orb', 'fg3a']","['median', 'avg', 'max']",0.188075,"['def_rtg', 'days_since_last_match', 'stl', 'fta_per_fga_pct', 'fg3a', 'trb', 'trb_pct', 'fg3_pct', 'orb_pct']","['max', 'avg']",1,1,0.669071
4,0.004734,0.000636,"[75, 30, 3, 1, 7]",1450,1.0,"['usg_pct', 'trb', 'pf', 'ast', 'drb', 'tov', 'ts_pct', 'fg3a_per_fga_pct', 'fga', 'blk', 'def_rtg', 'fg3', 'ft', 'fta_per_fga_pct', 'ft_pct', 'orb_pct', 'fta', 'stl_pct', 'ast_pct', 'pts', 'stl', 'fg_pct', 'blk_pct', 'fg', 'fg3a', 'efg_pct', 'plus_minus', 'drb_pct', 'mp']","['skew', 'min', 'max', 'median', 'avg']",0.187535,"['fg3_pct', 'tov_pct']",['avg'],1,1,0.668620
0,0.004509,0.002103,"[50, 5, 7]",1648,NaN,"['efg_pct', 'drb', 'fg3a_per_fga_pct', 'fg3', 'tov', 'fga', 'off_rtg', 'trb_pct', 'ast_pct', 'ast', 'fta_per_fga_pct', 'stl_pct', 'fg3_pct', 'orb', 'orb_pct', 'fg', 'ft_pct', 'plus_minus', 'blk_pct', 'ts_pct', 'trb', 'fg3a', 'pf']","['min', 'skew', 'max', 'avg']",0.189001,['drb'],"['median', 'max', 'avg']",1,1,0.668395
8,0.004509,0.000214,"[50, 30, 1, 75, 7]",450,1.0,"['mp', 'efg_pct', 'stl', 'blk_pct', 'fg_pct', 'orb', 'tov_pct', 'fg3a_per_fga_pct', 'trb', 'tov', 'ast', 'plus_minus', 'fg3a', 'blk', 'ft', 'fg3_pct', 'fta', 'drb_pct', 'off_rtg', 'stl_pct', 'drb', 'pf']","['var', 'median']",0.187113,"['win', 'blk', 'score_diff', 'fg_pct', 'ts_pct', 'fg3a_per_fga_pct', 'fta']",['avg'],1,1,0.668395
5,0.004509,0.002122,"[3, 7, 75, 1]",598,1.0,"['orb', 'off_rtg', 'mp', 'tov_pct', 'fg_pct', 'orb_pct', 'stl', 'trb', 'ft_pct', 'plus_minus', 'pts', 'ft', 'drb_pct', 'blk_pct', 'efg_pct', 'fg3', 'ast_pct', 'fg', 'def_rtg', 'fta', 'fta_per_fga_pct']","['avg', 'va

Accuracy baseline: 0.6638863841298467

In [12]:

df.sort_values('_outcome_accuracy_difference_from_baseline', ascending = False).head(20)


,_outcome_accuracy_difference_from_baseline,_score_r2_difference_from_baseline,aggregation_windows,col_count,manual_entry,player_aggregate_stat,player_aggregation_functions,points_estimation_r2,team_aggregate_stat,team_aggregation_functions,use_player_data,use_team_data,win_loss_accuracy
0,0.009017,-0.000597,"[40, 50, 1, 7, 30, 3, 75]",1256,1.0,"['tov', 'fg3a', 'fg3a_per_fga_pct', 'trb', 'stl', 'blk', 'def_rtg', 'ts_pct', 'plus_minus', 'drb_pct', 'off_rtg', 'fg3', 'fg', 'orb_pct', 'fta']","['median', 'max', 'var', 'skew', 'avg', 'min']",0.186301,"['fg', 'usg_pct', 'fg3', 'ast_pct', 'days_since_last_match', 'efg_pct', 'tov', 'fg3_pct', 'fta', 'blk', 'fga', 'ast', 'drb_pct', 'win', 'pf', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'stl', 'fg_pct', 'home', 'trb_pct', 'fg3a', 'tov_pct', 'orb', 'orb_pct', 'off_rtg']",['avg'],1,1,0.672904
0,0.008115,-0.000423,"[75, 7]",366,1.0,"['drb', 'ast', 'trb_pct', 'blk_pct', 'drb_pct', 'stl_pct', 'fg3', 'fg3a', 'fg3_pct', 'orb_pct', 'ts_pct', 'fta', 'off_rtg', 'plus_minus', 'tov_pct', 'ast_pct', 'ft_pct', 'tov']","['avg', 'min', 'max', 'var', 'skew']",0.186476,"['fg3_pct', 'efg_pct', 'fga', 'fg3a_per_fga_pct', 'stl_pct', 'fg_pct', 'pf', 'drb_pct', 'days_since_last_match', 'fta', 'tov_pct', 'orb_pct', 'fg3a', 'fg3', 'ft_pct', 'win', 'orb', 'blk_pct', 'ft', 'ast', 'def_rtg', 'usg_pct', 'ast_pct', 'off_rtg', 'drb', 'trb_pct', 'ts_pct']",['avg'],1,1,0.672002
1,0.006763,0.001035,"[1, 7, 75]",460,1.0,"['tov_pct', 'tov', 'usg_pct', 'orb', 'ast_pct', 'blk_pct', 'ts_pct', 'fg', 'pf', 'trb_pct', 'fg_pct', 'ft', 'mp', 'fg3_pct', 'fta_per_fga_pct', 'plus_minus', 'fta', 'orb_pct', 'ast']","['avg', 'var', 'min', 'skew']",0.187934,"['blk_pct', 'ft', 'fta_per_fga_pct', 'home', 'trb_pct']",['avg'],1,1,0.670649
1,0.006312,0.001019,"[7, 1, 75]",622,1.0,"['mp', 'plus_minus', 'blk', 'fta_per_fga_pct', 'fg3a_per_fga_pct', 'stl', 'pf', 'orb', 'trb_pct', 'pts', 'ast_pct', 'fga', 'ast', 'off_rtg', 'ts_pct', 'trb', 'fg3', 'usg_pct', 'efg_pct', 'fg3_pct', 'stl_pct', 'blk_pct', 'fg_pct', 'drb_pct', 'drb', 'fg3a', 'ft', 'fta', 'tov_pct', 'def_rtg', 'orb_pct', 'tov', 'fg', 'ft_pct']","['avg', 'var', 'skew']",0.187917,"['pts', 'fta', 'tov_pct', 'fg3_pct', 'fg', 'ts_pct', 'fg_pct', 'efg_pct', 'ast', 'orb', 'orb_pct', 'trb_pct', 'home', 'tov', 'blk', 'off_rtg', 'def_rtg', 'pf', 'blk_pct', 'score_diff', 'fta_per_fga_pct', 'drb']",['avg'],1,1,0.670198
2,0.005861,-0.001108,"[3, 75, 30, 7, 5, 1]",2408,1.0,"['plus_minus', 'stl_pct', 'fta', 'blk', 'ast_pct', 'pts', 'drb', 'tov', 'efg_pct', 'fg3', 'fg', 'orb_pct', 'orb', 'ft', 'ast', 'fta_per_fga_pct', 'fg3a', 'ft_pct', 'tov_pct', 'off_rtg', 'trb', 'blk_pct']","['var', 'median', 'min', 'max', 'avg']",0.185791,"['fg3_pct', 'fg3a']","['avg', 'skew']",1,1,0.669748
2,0.005861,-0.005851,"[7, 75]",754,1.0,"['fta_per_fga_pct', 'fg3a_per_fga_pct', 'usg_pct', 'ast_pct', 'ft', 'ast', 'blk', 'tov', 'pf', 'pts', 'drb_pct', 'fg3', 'orb_pct', 'fg3_pct', 'orb', 'stl', 'fga', 'efg_pct', 'blk_pct', 'fg_pct', 'mp', 'fg3a', 'tov_pct', 'plus_minus', 'trb_pct', 'ts_pct', 'def_rtg', 'drb', 'stl_pct', 'off_rtg', 'fta']","['var', 'skew', 'avg']",0.181047,"['fg3_pct', 'pf', 'blk', 'fg3a', 'blk_pct', 'stl_pct', 'off_rtg', 'def_rtg', 'stl', 'ft', 'score_diff', 'usg_pct', 'ast', 'orb', 'trb_pct', 'drb', 'fta', 'home', 'fta_per_fga_pct', 'ts_pct', 'efg_pct', 'fg_pct', 'orb_pct', 'win', 'drb_pct', 'trb', 'ft_pct', 'fga']","['max', 'avg']",1,1,0.669748
0,0.005410,0.002115,"[7, 1, 75, 30, 40]",1858,1.0,"['fg3a_per_fga_pct', 'trb_pct', 'pts', 'pf', 'fg3_pct', 'orb', 'fg3', 'off_rtg', 'plus_minus', 'ts_pct', 'fg3a']","['max', 'var', 'avg', 'skew']",0.189013,"['off_rtg', 'drb', 'pf', 'fg', 'tov', 'efg_pct', 'win', 'fta', 'trb_pct']","['max', 'avg', 'min', 'skew', 'median']",1,1,0.669297
3,0.005410,-0.001030,"[7, 3, 75, 1]",1362,1.0,"['blk_pct', 'stl_pct', 'tov', 'orb', 'plus_minus', 'fg3a', 'mp', 'fga', 'efg_pct', 'pf', 'ast_pct', 'ts_pct', 'fg3a_per_fga_pct', 'def_rtg', 'orb_pct', 'fg_pct', 'off_rtg', 'drb', 'fg3', 'tov_

R2 baseline: 0.18689847911073687

In [13]:

df.sort_values('_score_r2_difference_from_baseline', ascending = False).head(20)


,_outcome_accuracy_difference_from_baseline,_score_r2_difference_from_baseline,aggregation_windows,col_count,manual_entry,player_aggregate_stat,player_aggregation_functions,points_estimation_r2,team_aggregate_stat,team_aggregation_functions,use_player_data,use_team_data,win_loss_accuracy
80,0.000676,0.006156,"[5, 50, 3, 1, 75, 100, 7]",790,1.0,"['fg3a_per_fga_pct', 'ft', 'orb_pct', 'stl', 'fg', 'tov', 'efg_pct', 'ft_pct', 'stl_pct', 'fga', 'fg3a', 'trb', 'tov_pct', 'def_rtg', 'plus_minus']","['var', 'avg']",0.193054,['fg3_pct'],"['median', 'avg']",1,1,0.664563
49,0.002254,0.005151,"[7, 40, 3, 1, 75]",690,1.0,"['fg3a_per_fga_pct', 'ft_pct', 'stl', 'ast_pct', 'drb', 'tov', 'orb_pct', 'fg', 'fga', 'fg3_pct', 'blk', 'drb_pct', 'tov_pct', 'stl_pct', 'plus_minus', 'fta', 'off_rtg', 'efg_pct', 'ts_pct', 'blk_pct', 'ast', 'ft', 'trb']","['avg', 'min', 'max']",0.192050,"['blk', 'off_rtg', 'fg_pct', 'efg_pct', 'stl', 'days_since_last_match', 'ts_pct', 'fta_per_fga_pct', 'win', 'blk_pct', 'score_diff', 'fg3a_per_fga_pct', 'ft_pct', 'usg_pct', 'tov_pct', 'def_rtg', 'ast_pct', 'home', 'fg3', 'tov', 'fga', 'orb', 'trb_pct', 'fg3_pct', 'fg', 'drb', 'pf', 'stl_pct', 'fg3a', 'ft', 'pts', 'ast', 'orb_pct']",['avg'],1,1,0.666141
423,-0.002029,0.005089,"[7, 75, 50, 3, 1]",1544,1.0,"['pf', 'orb', 'fta_per_fga_pct', 'fg3a', 'drb', 'trb_pct', 'fg3', 'fg3_pct', 'ast', 'fg', 'fg_pct', 'stl_pct', 'fga', 'fg3a_per_fga_pct', 'mp', 'orb_pct', 'def_rtg', 'ft', 'plus_minus', 'ts_pct', 'off_rtg', 'blk_pct', 'stl', 'drb_pct', 'usg_pct', 'fta', 'pts', 'blk', 'tov_pct', 'trb']","['avg', 'min']",0.191988,"['trb_pct', 'off_rtg', 'fg', 'pts', 'orb_pct', 'ft_pct', 'fta_per_fga_pct', 'tov', 'pf', 'fg3a', 'ast', 'score_diff', 'efg_pct', 'home', 'days_since_last_match', 'drb', 'win', 'orb', 'stl', 'fg3_pct', 'drb_pct', 'trb', 'blk', 'ast_pct', 'ts_pct', 'fg3a_per_fga_pct', 'tov_pct', 'usg_pct']","['min', 'avg', 'median']",1,1,0.661858
737,-0.006087,0.004804,"[5, 7, 1, 3, 75, 40]",2278,1.0,"['drb_pct', 'fg3', 'fg3a_per_fga_pct', 'pf', 'fta', 'pts', 'ft', 'def_rtg', 'efg_pct', 'plus_minus', 'ft_pct']","['median', 'avg', 'min', 'skew', 'max']",0.191702,"['fg', 'win', 'fta', 'blk', 'fg_pct', 'orb', 'blk_pct', 'trb', 'fg3', 'fg3a_per_fga_pct', 'def_rtg', 'fg3a', 'ft_pct', 'efg_pct', 'ts_pct', 'fta_per_fga_pct', 'tov', 'tov_pct', 'trb_pct', 'stl_pct', 'drb', 'fg3_pct', 'stl', 'pts', 'score_diff', 'ast_pct', 'pf', 'home', 'usg_pct', 'drb_pct', 'off_rtg', 'days_since_last_match', 'orb_pct']","['min', 'skew', 'avg', 'max']",1,1,0.657800
627,-0.003607,0.004785,"[50, 75, 3, 7, 1, 30]",2152,1.0,"['plus_minus', 'def_rtg', 'stl_pct', 'fg3_pct', 'trb', 'fg3a_per_fga_pct', 'stl', 'drb', 'usg_pct', 'pts', 'ast', 'fg3', 'fg']","['median', 'min', 'max', 'avg']",0.191683,"['tov_pct', 'orb', 'home', 'fga', 'score_diff', 'stl', 'fg3', 'fg', 'drb', 'fta', 'trb', 'pf', 'ts_pct', 'orb_pct', 'off_rtg']","['skew', 'min', 'avg', 'max']",1,1,0.660280
140,-0.001127,0.004518,"[3, 7, 50, 1]",1330,1.0,"['drb', 'ast', 'mp', 'trb_pct', 'tov_pct', 'ft_pct', 'usg_pct', 'ts_pct', 'orb', 'fg3a', 'tov', 'fg3_pct', 'fta', 'def_rtg', 'drb_pct', 'fg_pct', 'fg3', 'blk', 'orb_pct', 'ft', 'blk_pct', 'pts', 'stl_pct', 'trb', 'plus_minus', 'fg3a_per_fga_pct', 'fg', 'off_rtg', 'pf', 'efg_pct', 'fga', 'fta_per_fga_pct', 'ast_pct']","['avg', 'var', 'max', 'median', 'skew']",0.191417,"['ts_pct', 'drb', 'stl', 'fg3a', 'pf', 'efg_pct', 'off_rtg', 'ast_pct', 'stl_pct', 'pts', 'ast', 'orb_pct', 'orb', 'blk', 'days_since_last_match', 'fg3a_per_fga_pct', 'fg_pct', 'usg_pct', 'home', 'score_diff', 'fga', 'tov_pct', 'fg', 'fg3_pct', 'ft', 'trb', 'tov', 'drb_pct', 'fta', 'ft_pct']",['avg'],1,1,0.662759
353,-0.003381,0.004439,"[20, 3, 50, 100, 1, 30, 5]",3668,1.0,"['fg3', 'fg', 'ast_pct', 'fg3a_per_fga_pct', 'orb_pct', 'drb_pct', 'plus_minus', 'blk_pct', 'stl', 'fg3_pct', 'pts', 'fta']","['avg', 'max', 'median', 'var', 'min']",0.191338,"['blk', 'tov', 'days_since_last_match', 'home', 'fg3_pct', 'fta', 'orb', 'efg_pct', 'win', 'f

In [14]:


def ohe(df_x, col_name):
    value_list = df_x[col_name].apply(lambda x: eval(x)).tolist()
    unique_values = set()

    for i in value_list:
        unique_values.update(set(i))

    unique_values = sorted(list(set(unique_values)))
#     eval_col = eval(x)

    for i in unique_values:
        df_x['{0}_{1}'.format(col_name, i)] = df_x[col_name].apply(lambda x: 1 if i in eval(x) else 0)
    df_x = df_x.drop(col_name, axis = 1)
    return df_x
    

In [15]:

def add_interactions(df):
    df_c = df.copy()

    for n, i1 in enumerate(df_c.columns):
        print(n, i1)
        for i2 in tqdm.tqdm(df_c.columns):
            if i1 == i2:
                continue
            df[f'prod_of_{i1}_{i2}'] = df[i1] * df[i2]
            
            if 'aggregate_stat' not in i1:
                for i3 in df_c.columns:
                    if i1 == i3 or i2 == i3:
                        continue
                    df[f'prod_of_{i1}_{i2}_{i3}'] = df[i1] *  df[i2] * df[i3]
    return df



def add_interactions_sklearn(df, df2, degree):
    poly = preprocessing.PolynomialFeatures(degree)
    df_poly = poly.fit_transform(df)
    df2_poly = poly.transform(df2)
    return df_poly, df2_poly


In [16]:

df_ohe = df.copy()
df_ohe = ohe(df_ohe, 'aggregation_windows')
df_ohe = ohe(df_ohe, 'player_aggregate_stat')
df_ohe = ohe(df_ohe, 'player_aggregation_functions')
df_ohe = ohe(df_ohe, 'team_aggregate_stat')
df_ohe = ohe(df_ohe, 'team_aggregation_functions')


df_x = df_ohe.drop(['manual_entry', '_outcome_accuracy_difference_from_baseline', '_score_r2_difference_from_baseline', 'points_estimation_r2', 'win_loss_accuracy'], axis = 1)



In [17]:
df_ohe[['win_loss_accuracy', 'points_estimation_r2']].describe()

,win_loss_accuracy,points_estimation_r2
count,5326.000000,5326.000000
mean,0.656355,0.179117
std,0.005453,0.007290
min,0.635708,0.156754
25%,0.652615,0.172829
50%,0.656447,0.181579
75%,0.660505,0.185328
max,0.672904,0.193054


In [18]:
# target_class = preprocessing.StandardScaler().fit_transform(df_ohe['win_loss_accuracy'].values.reshape(-1, 1)) 
# target_reg = preprocessing.StandardScaler().fit_transform(df_ohe['points_estimation_r2'].values.reshape(-1, 1)) 

In [19]:
target_class = df_ohe['win_loss_accuracy']
target_reg = df_ohe['points_estimation_r2']

In [20]:

df_rec_ohe = df_rec.copy()
df_rec_ohe = ohe(df_rec_ohe, 'aggregation_windows')
df_rec_ohe = ohe(df_rec_ohe, 'player_aggregate_stat')
df_rec_ohe = ohe(df_rec_ohe, 'player_aggregation_functions')
df_rec_ohe = ohe(df_rec_ohe, 'team_aggregate_stat')
df_rec_ohe = ohe(df_rec_ohe, 'team_aggregation_functions')


In [21]:
df_rec_ohe = df_rec_ohe[df_x.columns]


In [22]:
x_train, x_val, y_train1, y_val1, y_train2, y_val2 = model_selection.train_test_split(df_x, target_class, target_reg)

In [23]:
model1 = ensemble.RandomForestRegressor(n_jobs=-1, n_estimators = 10000)
model1.fit(x_train, y_train1)
metrics.r2_score(y_val1, model1.predict(x_val))



0.56453673359495

In [24]:
model2 = ensemble.RandomForestRegressor(n_jobs=-1, n_estimators = 10000)
model2.fit(x_train, y_train2)
metrics.r2_score(y_val2, model2.predict(x_val))


0.8610752218948826

In [25]:
df_x.shape, df_rec_ohe.shape

((5326, 96), (1000000, 96))

In [26]:
# df_x2, df_rec_ohe2 = add_interactions_sklearn(df_x, df_rec_ohe, 2)
# df_x2.shape, df_rec_ohe2.shape

In [27]:
# df_x3, df_rec_ohe3 = add_interactions_sklearn(df_x, df_rec_ohe, 3)
# df_x3.shape, df_rec_ohe3.shape

In [28]:
df_x.shape, df_rec_ohe.shape

((5326, 96), (1000000, 96))

In [29]:
model_class = ensemble.RandomForestRegressor(n_jobs=-1, n_estimators = 10000)
model_class.fit(df_x, target_class)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10000,
                      n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [30]:
model_reg = ensemble.RandomForestRegressor(n_jobs=-1, n_estimators = 10000)
model_reg.fit(df_x, target_reg)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10000,
                      n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [31]:
# model_class3 = ensemble.RandomForestRegressor(n_jobs=-1, max_features='log2', n_estimators = 100)
# model_class3.fit(df_x3, target_class)


In [32]:
# model_reg3 = ensemble.RandomForestRegressor(n_jobs=-1, max_features='log2', n_estimators = 100)
# model_reg3.fit(df_x3, target_reg)


In [33]:

df_rec['pred_accuracy'] = model_class.predict(df_rec_ohe)
df_rec['pred_r2'] = model_reg.predict(df_rec_ohe)
# df_rec['pred_accuracy_3'] = model_class3.predict(df_rec_ohe3)
# df_rec['pred_r2_3'] = model_reg3.predict(df_rec_ohe3)


In [34]:
df_rec['scaled_pred_accuracy'] = preprocessing.QuantileTransformer().fit_transform(df_rec['pred_accuracy'].values.reshape(-1, 1))
df_rec['scaled_pred_r2'] = preprocessing.QuantileTransformer().fit_transform(df_rec['pred_r2'].values.reshape(-1, 1))

df_rec['overall_score_1'] = 0
df_rec['overall_score_1'] += df_rec['scaled_pred_accuracy']
# df_rec['overall_score_1'] += df_rec['scaled_pred_r2']



In [35]:
df_rec.describe()

,col_count,use_player_data,use_team_data,pred_accuracy,pred_r2,scaled_pred_accuracy,scaled_pred_r2,overall_score_1
count,1000000.000000,1000000.0,1000000.0,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,1213.771592,1.0,1.0,0.652874,0.174543,0.500015,0.500739,0.500015
std,1033.644858,0.0,0.0,0.001834,0.004571,0.288692,0.288927,0.288692
min,12.000000,1.0,1.0,0.646059,0.163196,0.000000,0.000000,0.000000
25%,362.000000,1.0,1.0,0.651708,0.171046,0.249685,0.249787,0.249685
50%,898.000000,1.0,1.0,0.652563,0.174057,0.499670,0.501832,0.499670
75%,1850.000000,1.0,1.0,0.653710,0.177552,0.749995,0.749971,0.749995
max,4000.000000,1.0,1.0,0.664062,0.189070,1.000000,1.000000,1.000000


In [36]:
df_rec.sort_values('overall_score_1', ascending = False).head(5)


,col_count,use_player_data,use_team_data,aggregation_windows,player_aggregate_stat,player_aggregation_functions,team_aggregate_stat,team_aggregation_functions,pred_accuracy,pred_r2,scaled_pred_accuracy,scaled_pred_r2,overall_score_1
165882,1290,1,1,"[50, 7, 75, 3]","['fta_per_fga_pct', 'fga', 'drb_pct', 'blk_pct', 'trb', 'fg', 'plus_minus', 'fg3a_per_fga_pct', 'ft', 'pf', 'def_rtg', 'fg_pct', 'tov', 'fg3_pct', 'fta', 'ast_pct', 'fg3a', 'stl_pct', 'ft_pct', 'tov_pct', 'orb', 'orb_pct', 'trb_pct', 'blk', 'pts', 'mp', 'ast', 'efg_pct', 'stl', 'off_rtg', 'usg_pct', 'ts_pct']","['var', 'skew', 'median', 'max', 'avg']","['trb_pct', 'tov_pct', 'fga', 'usg_pct', 'fg', 'ast', 'pts', 'fta', 'off_rtg', 'home', 'stl_pct', 'pf', 'efg_pct', 'fg3a', 'tov', 'drb', 'win', 'ts_pct', 'def_rtg', 'days_since_last_match', 'ft', 'drb_pct', 'fg3_pct', 'fta_per_fga_pct', 'orb', 'stl', 'trb', 'fg3a_per_fga_pct', 'fg_pct', 'fg3', 'score_diff', 'ft_pct', 'blk', 'orb_pct', 'ast_pct', 'blk_pct']",['avg'],0.663682,0.186594,1.000000,0.997173,1.000000
512577,406,1,1,"[75, 7]","['fg', 'stl_pct', 'blk_pct', 'plus_minus', 'pts', 'drb', 'trb', 'ts_pct', 'ast', 'blk', 'ft', 'mp', 'fta_per_fga_pct', 'drb_pct', 'stl', 'ast_pct', 'tov_pct', 'trb_pct', 'tov', 'fg3']","['avg', 'min', 'skew', 'var', 'max']","['blk_pct', 'drb_pct']",['avg'],0.663629,0.185075,1.000000,0.983960,1.000000
874773,994,1,1,"[7, 3, 75]","['fg_pct', 'stl_pct', 'trb_pct', 'tov', 'fta', 'orb_pct', 'fg3', 'off_rtg', 'fg', 'tov_pct', 'fg3a_per_fga_pct', 'ast_pct', 'mp', 'pf', 'blk', 'def_rtg', 'stl', 'plus_minus', 'orb', 'ft_pct', 'drb_pct', 'fg3_pct', 'fg3a', 'ast', 'pts', 'ts_pct', 'trb', 'fga', 'drb', 'fta_per_fga_pct', 'blk_pct', 'usg_pct', 'efg_pct']","['avg', 'median', 'max', 'min', 'var']","['fg', 'blk', 'drb_pct', 'pts', 'fg_pct', 'trb', 'drb', 'fga', 'ft_pct', 'fg3a', 'off_rtg', 'trb_pct', 'fta_per_fga_pct', 'pf', 'days_since_last_match', 'score_diff', 'usg_pct', 'ts_pct', 'fg3_pct', 'def_rtg', 'blk_pct', 'ft', 'home', 'ast_pct', 'tov', 'orb', 'fg3', 'stl']",['avg'],0.664062,0.185285,1.000000,0.986613,1.000000
125023,706,1,1,"[7, 40, 75]","['tov_pct', 'blk', 'orb', 'ft', 'ast_pct', 'tov', 'off_rtg', 'fta_per_fga_pct', 'fg3a_per_fga_pct', 'pf', 'pts', 'ts_pct', 'drb', 'trb', 'ast', 'mp', 'fg3a', 'fg_pct', 'trb_pct', 'ft_pct', 'orb_pct', 'efg_pct', 'blk_pct', 'stl_pct', 'fg3_pct', 'plus_minus', 'fg3', 'drb_pct', 'def_rtg']","['var', 'avg', 'median', 'skew']","['ft_pct', 'drb', 'fg3a', 'fta_per_fga_pct', 'blk_pct', 'trb_pct', 'fg3', 'fta']",['avg'],0.663484,0.185034,0.999945,0.983344,0.999945
833718,726,1,1,"[75, 7]","['pf', 'usg_pct', 'fg3a', 'drb', 'off_rtg', 'fg3a_per_fga_pct', 'fga', 'stl', 'blk', 'tov_pct', 'plus_minus', 'fta_per_fga_pct', 'orb_pct', 'orb', 'ts_pct', 'ft', 'fg', 'drb_pct', 'ast_pct', 'def_rtg', 'fg_pct', 'fta', 'fg3', 'pts', 'blk_pct', 'efg_pct', 'tov', 'fg3_pct', 'ast', 'trb_pct']","['max', 'var', 'skew', 'avg', 'min', 'median']","['tov_pct', 'orb_pct', 'drb_pct', 'tov', 'pf', 'ts_pct', 'stl', 'days_since_last_match', 'ft', 'def_rtg', 'efg_pct', 'fg3', 'fg', 'fta', 'ast', 'score_diff', 'home', 'fga', 'ft_pct', 'trb_pct', 'orb', 'drb', 'fg_pct']",['avg'],0.663455,0.185044,0.999934,0.983500,0.999934


In [37]:
solutions = df_rec.sort_values('overall_score_1', ascending = False).head(1000).to_dict(orient = 'records')
solutions

[{'col_count': 1290,
  'use_player_data': 1,
  'use_team_data': 1,
  'aggregation_windows': '[50, 7, 75, 3]',
  'player_aggregate_stat': "['fta_per_fga_pct', 'fga', 'drb_pct', 'blk_pct', 'trb', 'fg', 'plus_minus', 'fg3a_per_fga_pct', 'ft', 'pf', 'def_rtg', 'fg_pct', 'tov', 'fg3_pct', 'fta', 'ast_pct', 'fg3a', 'stl_pct', 'ft_pct', 'tov_pct', 'orb', 'orb_pct', 'trb_pct', 'blk', 'pts', 'mp', 'ast', 'efg_pct', 'stl', 'off_rtg', 'usg_pct', 'ts_pct']",
  'player_aggregation_functions': "['var', 'skew', 'median', 'max', 'avg']",
  'team_aggregate_stat': "['trb_pct', 'tov_pct', 'fga', 'usg_pct', 'fg', 'ast', 'pts', 'fta', 'off_rtg', 'home', 'stl_pct', 'pf', 'efg_pct', 'fg3a', 'tov', 'drb', 'win', 'ts_pct', 'def_rtg', 'days_since_last_match', 'ft', 'drb_pct', 'fg3_pct', 'fta_per_fga_pct', 'orb', 'stl', 'trb', 'fg3a_per_fga_pct', 'fg_pct', 'fg3', 'score_diff', 'ft_pct', 'blk', 'orb_pct', 'ast_pct', 'blk_pct']",
  'team_aggregation_functions': "['avg']",
  'pred_accuracy': 0.6636821857202504,
  '

In [38]:
solutions_clean = []

for i in solutions:
#     print(list(i.keys()))
    
    next_dict = dict()
    next_dict['use_player_data'] = i['use_player_data']
    next_dict['use_team_data'] = i['use_team_data']
    next_dict['aggregation_windows'] = eval(i['aggregation_windows'])
    next_dict['player_aggregate_stat'] = eval(i['player_aggregate_stat'])
    next_dict['player_aggregation_functions'] = eval(i['player_aggregation_functions'])
    next_dict['team_aggregate_stat'] = eval(i['team_aggregate_stat'])
    next_dict['player_aggregation_functions'] = eval(i['player_aggregation_functions'])
    next_dict['team_aggregation_functions'] = eval(i['team_aggregation_functions'])
    next_dict['manual_entry'] = 1
    solutions_clean.append(next_dict)
    
solutions_clean

[{'use_player_data': 1,
  'use_team_data': 1,
  'aggregation_windows': [50, 7, 75, 3],
  'player_aggregate_stat': ['fta_per_fga_pct',
   'fga',
   'drb_pct',
   'blk_pct',
   'trb',
   'fg',
   'plus_minus',
   'fg3a_per_fga_pct',
   'ft',
   'pf',
   'def_rtg',
   'fg_pct',
   'tov',
   'fg3_pct',
   'fta',
   'ast_pct',
   'fg3a',
   'stl_pct',
   'ft_pct',
   'tov_pct',
   'orb',
   'orb_pct',
   'trb_pct',
   'blk',
   'pts',
   'mp',
   'ast',
   'efg_pct',
   'stl',
   'off_rtg',
   'usg_pct',
   'ts_pct'],
  'player_aggregation_functions': ['var', 'skew', 'median', 'max', 'avg'],
  'team_aggregate_stat': ['trb_pct',
   'tov_pct',
   'fga',
   'usg_pct',
   'fg',
   'ast',
   'pts',
   'fta',
   'off_rtg',
   'home',
   'stl_pct',
   'pf',
   'efg_pct',
   'fg3a',
   'tov',
   'drb',
   'win',
   'ts_pct',
   'def_rtg',
   'days_since_last_match',
   'ft',
   'drb_pct',
   'fg3_pct',
   'fta_per_fga_pct',
   'orb',
   'stl',
   'trb',
   'fg3a_per_fga_pct',
   'fg_pct',
   'fg3',

In [39]:
import json

with open('/tmp/next_params.json', 'w') as f:
    json.dump(solutions_clean, f)

In [40]:
len(solutions_clean)

1000

In [41]:
df = pd.read_csv()

TypeError: parser_f() missing 1 required positional argument: 'filepath_or_buffer'